In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader

import numpy as np

In [2]:
labels = np.load("labels.npy")

training_generator = DataLoader(labels, 128)

In [ ]:
model = Sequential()
model.add(Dense(768, activation='relu', input_shape=(98304,)))
# model.add(Dense(768, activation='relu'))
model.add(Dense(768, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.fit(training_generator, epochs=1)

2022-02-26 14:37:58.300947: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 14:37:58.653288: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 14:37:58.768889: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


    4/10457 [..............................] - ETA: 13:54 - loss: 6.6211 - accuracy: 0.0098     

2022-02-26 14:37:58.978429: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 14:37:59.082116: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


10457/10457 [==============================] - 896s 86ms/step - loss: 4.7113 - accuracy: 0.1061


In [7]:
import time
model.save("models/model-{}".format(int(time.time())))

INFO:tensorflow:Assets written to: models/model-1645905292/assets
